In [46]:
# import secrets and tokens from config.py
from config import client_id, client_secret, refresh_token

import requests
import urllib3

# import packages for data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': "refresh_token",
    'f': 'json'
}

In [47]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

Requesting Token...

Access Token = fc4f2b5708a67b512d1122f2673e22c2a960d9e2



In [48]:
activities = pd.json_normalize(my_dataset)

In [49]:
#Break date into start time and date
activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])
activities['start_time'] = activities['start_date_local'].dt.time
activities['start_date_local'] = activities['start_date_local'].dt.date

In [50]:
# import modules
import os
import time
import matplotlib.pyplot as plt
import folium
import polyline
import base64
from tqdm import tqdm

In [79]:
# df.query('column_name.str.contains("abc")', engine='python')
activities_nfs_copy = activities.copy(deep = True)
# the line below is likely causing the "A value is trying to be set on a copy of a slice from a DataFrame." warning.
activities_nfs = activities_nfs_copy.query('name.str.contains("NFS")', engine = 'python')
activities_nfs.reset_index(drop = True, inplace = True)
len(activities_nfs)

4

In [80]:
# add decoded summary polylines
# activities['map.summary_polyline'] contains an encoded polyline
# .apply(polyline.decode) decodes that polyline into latitude and longitude
activities_nfs['map.polyline'] = activities_nfs['map.summary_polyline'].apply(polyline.decode)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_14532\329265630.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities_nfs['map.polyline'] = activities_nfs['map.summary_polyline'].apply(polyline.decode)


In [71]:
activities_nfs['map.summary_polyline'].head(3)

0    kgjjF~hrdVkP{FcCs@SBGN]~BEHsBg@I@wBvLMX_@VIRq@...
1    unjjFnerdVsMgE_Ck@qDfSKPY@M^o@pEO`@iGz^}CdQc@|...
2    aijjFxgrdVaD_A{Q_Ge@IONkBpLkA~FcDdRwEbXgCbOS`A...
Name: map.summary_polyline, dtype: object

In [72]:
activities_nfs['map.polyline'][0][0]

(38.56006, -121.50432)

In [73]:
my_first_ride = activities_nfs.iloc[0, :]
my_first_ride

resource_state                                                          2
name                                                        NFS Team Ride
distance                                                          32270.1
moving_time                                                          5542
elapsed_time                                                         6274
                                              ...                        
map.summary_polyline    kgjjF~hrdVkP{FcCs@SBGN]~BEHsBg@I@wBvLMX_@VIRq@...
map.resource_state                                                      2
average_temp                                                          NaN
start_time                                                       06:20:42
map.polyline            [(38.56006, -121.50432), (38.56284, -121.50306...
Name: 0, Length: 62, dtype: object

In [74]:
activities_nfs['map.polyline'][0][0]

(38.56006, -121.50432)

In [81]:
type(my_first_ride)

pandas.core.series.Series

In [78]:
len(activities_nfs)

4

In [84]:
activities_nfs['map.polyline'][0][0]

(38.56006, -121.50432)

In [96]:
m = folium.Map(location=activities_nfs['map.polyline'][0][0], zoom_start=12.25)
counter = 0

while counter < len(activities_nfs):
    
    ride = activities_nfs.iloc[counter, :]
    folium.PolyLine(ride['map.polyline'], color = 'red').add_to(m)
    counter+=1
    
display(m)

In [76]:
# select one activity
my_first_ride = activities_nfs.iloc[0, :] # first activity (most recent)
my_second_ride = activities_nfs.iloc[1, :] # first activity (most recent)
my_third_ride = activities_nfs.iloc[2, :] # first activity (most recent)
my_fourth_ride = activities_nfs.iloc[3, :] # first activity (most recent)
# plot ride on map

m = folium.Map(location=activities_nfs['map.polyline'][0][0], zoom_start=14)
folium.PolyLine(my_first_ride['map.polyline'], color='red').add_to(m)
folium.PolyLine(my_second_ride['map.polyline'], color='red').add_to(m)
folium.PolyLine(my_third_ride['map.polyline'], color='red').add_to(m)
folium.PolyLine(my_fourth_ride['map.polyline'], color='red').add_to(m)

display(m)

In [ ]:
# NOT WORKING RIGHT NOW B/C THE ELEVATION API IS DOWN #
#####################
# define function to get elevation data using the open-elevation API
def get_elevation(latitude, longitude):
    base_url = 'https://api.open-elevation.com/api/v1/lookup'
    payload = {'locations': f'{latitude},{longitude}'}
    r = requests.get(base_url, params=payload).json()['results'][0]
    return r['elevation']
# get elevation data
elevation_data = list()
for idx in tqdm(activities_nfs.index):
    activity = activities_nfs.loc[idx, :]
    elevation = [get_elevation(coord[0], coord[1]) for coord in activity['map.polyline']]
    elevation_data.append(elevation)

  0%|          | 0/4 [01:01<?, ?it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# select one activity
my_ride = activities.iloc[0, :] # first activity (most recent)
# plot ride on map
centroid = [
    np.mean([coord[0] for coord in my_ride['map.polyline'][0]]), 
    np.mean([coord[1] for coord in my_ride['map.polyline'][0]])
]
m = folium.Map(location=centroid, zoom_start=14)
folium.PolyLine(my_ride['map.polyline'], color='red').add_to(m)
display(m)

In [ ]:
# select one activity
my_ride = activities.iloc[0, :] # first activity (most recent)
# plot ride on map
# centroid = [
#     np.mean([coord[0] for coord in my_ride['map.polyline'][0]]), 
#     np.mean([coord[1] for coord in my_ride['map.polyline'][0]])
# ]
m = folium.Map(location=activities['map.polyline'][0][0], zoom_start=14)
folium.PolyLine(my_ride['map.polyline'], color='red').add_to(m)
display(m)

In [16]:
# add decoded summary polylines
activities['map.polyline'] = activities['map.summary_polyline'].apply(polyline.decode)

In [17]:

#####################
# define function to get elevation data using the open-elevation API
def get_elevation(latitude, longitude):
    base_url = 'https://api.open-elevation.com/api/v1/lookup'
    payload = {'locations': f'{latitude},{longitude}'}
    r = requests.get(base_url, params=payload).json()['results'][0]
    return r['elevation']
# get elevation data
elevation_data = list()
for idx in tqdm(activities.index):
    activity = activities.loc[idx, :]
    elevation = [get_elevation(coord[0], coord[1]) for coord in activity['map.polyline']]
    elevation_data.append(elevation)

  0%|          | 0/200 [01:04<?, ?it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [15]:

#####################
# define function to get elevation data using the open-elevation API
def get_elevation(latitude, longitude):
    base_url = 'https://api.open-elevation.com/api/v1/lookup'
    payload = {'locations': f'{latitude},{longitude}'}
    r = requests.get(base_url, params=payload).json()['results'][0]
    return r['elevation']
# get elevation data
elevation_data = list()
for idx in tqdm(activities_nfs.index):
    activity = activities_nfs.loc[idx, :]
    elevation = [get_elevation(coord[0], coord[1]) for coord in activity['map.polyline']]
    elevation_data.append(elevation)

  0%|          | 0/4 [01:01<?, ?it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
# Make a copy of activities DataFrame for testing.
activities_copy = activities.copy()

In [6]:
# convert data types
activities_copy.loc[:, 'start_date'] = pd.to_datetime(activities_copy['start_date']).dt.tz_localize(None)
activities_copy.loc[:, 'start_date_local'] = pd.to_datetime(activities_copy['start_date_local']).dt.tz_localize(None)
# convert values
activities_copy.loc[:, 'distance'] /= 1609.344 # convert from meters to miles
activities_copy.loc[:, 'average_speed'] *= 2.23693629 # convert from meters/second to miles/hour
activities_copy.loc[:, 'max_speed'] *= 2.23693629 # convert from meters/second to miles/hour
# set index
activities_copy.set_index('start_date_local', inplace=True)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_2416\1035570082.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  activities_copy.loc[:, 'start_date'] = pd.to_datetime(activities_copy['start_date']).dt.tz_localize(None)
C:\Users\jimmy\AppData\Local\Temp\ipykernel_2416\1035570082.py:3: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  activities_copy.loc[:, 'start_date_local'] = pd.to_datetime(activities_copy['start_date_local']).dt.tz_localize(None)


In [12]:
activities_nfs['map.polyline'].head(2)


9     [(38.56006, -121.50432), (38.56284, -121.50306...
16    [(38.56123, -121.50376), (38.56357, -121.50276...
Name: map.polyline, dtype: object